In [ ]:
from TraceLens import TreePerfAnalyzer
from TraceLens.util import DataLoader

In [ ]:
from TraceLens.Trace2Tree.trace_to_tree import JaxTraceToTree
from TraceLens.util import TraceEventUtils

In [ ]:

profile_filepath = "/workspace/code/traces/logs/20250509_1604_mixtral/tw026/20250509_1604_mixtral/tensorboard/plugins/profile/2025_05_09_16_09_12/tw026.xplane.pb"

#data_pb = DataLoader.load_data(profile_filepath, save_preprocessed=True)
data_pb = DataLoader.load_data(profile_filepath, save_preprocessed=False)


In [ ]:
data_pb.keys()

In [ ]:
data_pb["showCounterMessage"]

In [ ]:
data_pb = data_pb['traceEvents']

In [ ]:
jax = True
linking_key = 'correlation_id'
categorizer =  JaxTraceToTree.default_categorizer if not jax else TraceEventUtils.prepare_event_categorizer(data_pb)

data = data_pb if not jax else TraceEventUtils.non_metadata_events(data_pb)

tree = JaxTraceToTree(data, event_to_category=categorizer, linking_key=linking_key)
metadata = TraceEventUtils.get_metadata(data_pb)
tree.build_tree(metadata=metadata, pb_file_name=profile_filepath)

In [ ]:
for event in tree.events:
    if "args" in event.keys():
        if "hlo_op" in event["args"].keys():
            if event["args"].get("hlo_op") == "all-gather-start.10":
                print(event)
                

In [ ]:
metadata = TraceEventUtils.get_metadata(data_pb)

In [ ]:
metadata.keys()

In [ ]:
metadata[1]

In [ ]:
tree._set_metadata(metadata=metadata)
tree._set_hlo_ops(pb_file_name=profile_filepath)

In [ ]:
tree._create_linking_key_to_uid_map()

In [ ]:
len(tree.linking_key_to_uid_map.keys())

In [ ]:
tree.build_tree(metadata=metadata, pb_file_name=profile_filepath)

In [ ]:
ls = tree.hlo_ops["jit_train_step"]

In [ ]:
type(ls)

In [ ]:
ls["%all-gather-start.29"]

In [ ]:
keys = list(ls.keys())

In [ ]:
len(keys)

In [ ]:
tree.linking_key

In [ ]:
for ind in range(len(tree.events)):
    if tree.events[ind].get('gpu_kernel_op_cat',"NA") == "Communication rccl/nccl":
        print(f"Index : {ind}")
        print(tree.events[ind])
        break
    

In [ ]:
tree.events[83]

In [ ]:
from tensorboard_plugin_profile.convert import raw_to_tool_data as convert
import itertools
import json
import os
import re
import glob

In [ ]:
protobuf_file_name = "/workspace/code/traces/logs/20250509_1604_mixtral/tw026/20250509_1604_mixtral/tensorboard/plugins/profile/2025_05_09_16_09_12/tw026.xplane.pb"

In [ ]:
dir_name = os.path.dirname(protobuf_file_name) + "/"
print(dir_name)
module_name = "jit_train_step"

In [ ]:
print(f"{dir_name}{os.path.sep}{module_name}*hlo_proto.pb")

In [ ]:
hlo_filename = glob.glob(dir_name + os.path.sep + module_name + "*hlo_proto.pb")
hlo_filename

In [ ]:
hlo_filename = glob.glob(dir_name + os.path.sep + module_name + "*hlo_proto.pb")
if len(hlo_filename) != 1:
    convert.xspace_to_tool_names([protobuf_file_name])
hlo_filename = glob.glob(dir_name + os.path.sep + module_name + "*hlo_proto.pb")
#assert len(hlo_filename) == 0
if len(hlo_filename) > 1:
    print('Multiple matching hlo_filenames:')
    print(hlo_filename)
elif len(hlo_filename) == 0:
    print('No matching hlo_filenames:')
    print(hlo_filename)


In [ ]:

# need to make sure that the pb exists and get the numerical suffix into the module name
# and remove '.hlo_proto.pb'
module_name = os.path.splitext(os.path.splitext(os.path.basename(hlo_filename[0]))[0])[0]
module_name


In [ ]:

hlo_ops={}
graph_viewer_options= {
    'node_name': "",
    'module_name': module_name,
    'graph_width': 2,
    'show_metadata': False,
    'merge_fusion': False,
    'type': "long_txt"
}
params = {'graph_viewer_options': graph_viewer_options }
data, _ = convert.xspace_to_tool_data(
        [dir_name], "graph_viewer^", params)
data = data.decode("utf-8").split('\n')

In [ ]:
len(data)

In [ ]:
data[2]

In [ ]:
for ind in range(len(data)):
    if "all-to-all.225.1" in data[ind]:
        print(data[ind])
        break

In [ ]:
for line in data:
    line_processed=line.strip()
    if (("metadata" in line_processed and not(re.search(r"\)$",line_processed)) and not(line_processed.startswith("ROOT")))
            or any(t in line_processed for t in ["get-tuple-element", "bf16", "f8", "f16", "f32", "f64"])
            and not(line_processed.startswith("HloModule "))):
            if "replica_groups" in line_processed:
                print(line)
                break
            # print(line)
            # line=re.sub(r"\),",")",line)
            # print(line)
            # line=re.sub(r", ",",",line)
            # print(line)
            # line=re.sub(r" %","%",line)
            # print(line)

In [ ]:
line = '%all-gather-start.28 = ((bf16[1,768,48,128]{3,2,0,1}, bf16[1,768,8,128]{3,2,0,1}, bf16[1,768,8,128]{3,2,0,1}), (bf16[1,6144,48,128]{3,2,0,1}, bf16[1,6144,8,128]{3,2,0,1}, bf16[1,6144,8,128]{3,2,0,1})) all-gather-start(%get-tuple-element.24226, %get-tuple-element.24230, %get-tuple-element.24231), channel_id=5, replica_groups={{0,1,2,3,4,5,6,7},{8,9,10,11,12,13,14,15},{16,17,18,19,20,21,22,23},{24,25,26,27,28,29,30,31}}, dimensions={1}, use_global_device_ids=true, metadata={scheduling_name="all-gather-start.28"}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"collective_backend_config":{"is_sync":false,"no_parallel_custom_call":true,"is_pipelined":false},"force_earliest_schedule":false}'
print(line)
line=re.sub(r"\),",")",line)
print(line)
line=re.sub(r", ",",",line)
print(line)
line=re.sub(r" %","%",line)
print(line)

In [ ]:
backend_config=re.search(r"backend_config=\{[a-zA-Z_=\"\(\)\/0-9\ @.-:,\[\]\{\}]*",line)
print(backend_config)
metadata=re.search(r"metadata=\{[a-zA-Z_=\"\(\)\/0-9\ @.-]*",line)
print(metadata)
custom_call_target=re.search(r"custom_call_target=\"[a-zA-Z_=\"\(\)\/0-9\ @.\-\$]*",line)
print(custom_call_target)
line=line.split(" ")
print(line)

In [ ]:
import argparse
import sys
from glob import glob
from pathlib import Path
import re
import gzip
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os
from tensorboard_plugin_profile.convert import raw_to_tool_data as convert
import os
import rccl_multinode_analysis as rma

In [ ]:
xla_dump = "/workspace/code/JaxTrace_Analysis/logs/20250509_1604_mixtral/tw026/xla_dumps"

In [ ]:
tmp = rma.detect_xla_file(xla_dump)
print(tmp)

In [ ]:
messages, replica_groups = rma.read_xla_dump(tmp)

In [ ]:
messages["all-gather-start.10"]

In [ ]:
replica_groups["all-gather-start.28"]